# NIH Chest X-ray Multi label Binary classification using Tensorflow Densenet121 (Transfer learning)


In [2]:
# Go to project root folder
import os
os.chdir("../")
%pwd

'/workspaces/chest-x-ray-diagnosis'

## Imports

In [3]:
# Set environment variables
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.random.set_seed(42)

2025-03-04 08:03:06.424150: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741075386.435442    2383 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741075386.439235    2383 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
found_gpu = tf.config.list_physical_devices('GPU')
if not found_gpu:
    raise Exception("No GPU found")
found_gpu, tf.__version__

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.18.0')

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pathlib import Path
import opendatasets as od

from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K


In [6]:
%matplotlib inline

# auto reload libs
%load_ext autoreload
%autoreload 2

## Constants

In [7]:

IMAGE_SIZE = 320
BATCH_SIZE = 16

## Download the dataset


In [8]:
dataset_url = 'https://www.kaggle.com/datasets/nih-chest-xrays/sample'

# Look into the data directory
datasets = 'datasets/sample'
dataset_path = Path(datasets)
IMAGE_DIR = dataset_path /'sample/images'
CSV_PATH = dataset_path /'sample/sample_labels.csv'
dataset_path.mkdir(parents=True, exist_ok=True)
if not dataset_path.is_dir():
    od.download(dataset_url)

## Paths Setup

In [9]:
from hydra import initialize, compose

# https://gist.github.com/bdsaglam/586704a98336a0cf0a65a6e7c247d248

with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config")
    print(cfg.DATASET_DIRS.TRAIN_IMAGES_DIR)

datasets/sample/sample/images


## Load Datasets

In [13]:
from utils.chest_x_ray_preprocessor import ChestXRayPreprocessor

preprocessor = ChestXRayPreprocessor(cfg)
train_ds, valid_ds, test_ds, pos_weights, neg_weights = preprocessor.get_preprocessed_datasets()

   Atelectasis  Effusion  Infiltration  Mass  Nodule
0            0         0             0     0       0
1            1         1             1     0       0
2            0         0             0     0       0
3            0         0             0     0       0
4            0         0             1     0       0


In [15]:
for batch in train_ds.take(1):
    images, labels = batch
    print(images.shape, labels.shape)
    print(images[0], labels[0])

(12, 320, 320, 3) (12, 15)
tf.Tensor(
[[[-1.0892599  -1.0892599  -1.0892599 ]
  [-1.1530176  -1.1530176  -1.1530176 ]
  [-1.3071464  -1.3071464  -1.3071464 ]
  ...
  [-0.23227446 -0.23227446 -0.23227446]
  [-0.2411862  -0.2411862  -0.2411862 ]
  [-0.25740695 -0.25740695 -0.25740695]]

 [[-0.9662309  -0.9662309  -0.9662309 ]
  [-0.9005775  -0.9005775  -0.9005775 ]
  [-0.95811623 -0.95811623 -0.95811623]
  ...
  [-0.23006429 -0.23006429 -0.23006429]
  [-0.24542373 -0.24542373 -0.24542373]
  [-0.26421484 -0.26421484 -0.26421484]]

 [[-0.8271123  -0.8271123  -0.8271123 ]
  [-0.75552446 -0.75552446 -0.75552446]
  [-0.71787786 -0.71787786 -0.71787786]
  ...
  [-0.2240171  -0.2240171  -0.2240171 ]
  [-0.23195426 -0.23195426 -0.23195426]
  [-0.25150064 -0.25150064 -0.25150064]]

 ...

 [[-0.72461647 -0.72461647 -0.72461647]
  [-0.68322694 -0.68322694 -0.68322694]
  [-0.6184415  -0.6184415  -0.6184415 ]
  ...
  [-0.15838882 -0.15838882 -0.15838882]
  [-0.21605061 -0.21605061 -0.21605061]
  [-0.

In [14]:
sample_df = pd.read_csv(CSV_PATH)
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5606 entries, 0 to 5605
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Image Index                  5606 non-null   object 
 1   Finding Labels               5606 non-null   object 
 2   Follow-up #                  5606 non-null   int64  
 3   Patient ID                   5606 non-null   int64  
 4   Patient Age                  5606 non-null   object 
 5   Patient Gender               5606 non-null   object 
 6   View Position                5606 non-null   object 
 7   OriginalImageWidth           5606 non-null   int64  
 8   OriginalImageHeight          5606 non-null   int64  
 9   OriginalImagePixelSpacing_x  5606 non-null   float64
 10  OriginalImagePixelSpacing_y  5606 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 481.9+ KB


In [ ]:
sample_df.columns

In [ ]:
valid_csv= 'datasets/valid-small.csv'
test_csv= 'datasets/test.csv'

valid_df = pd.read_csv(f"{valid_csv}")
test_df = pd.read_csv(f"{test_csv}")

In [ ]:
valid_df.head()

In [ ]:
valid_df.drop('PatientId', axis=1, inplace=True)
valid_df.head()

In [ ]:
len(valid_df.columns)

### Unique check

In [ ]:
sample_df['Patient ID'].count(),sample_df['Patient ID'].nunique()

## Data Clean up

Droping duplicates

In [ ]:
sample_df.drop_duplicates(subset=['Patient ID'], inplace=True)
sample_df['Patient ID'].nunique()


In [ ]:
sample_labels = ['Image Index', 'Finding Labels']
sample_df = sample_df[sample_labels]
sample_df.columns

In [ ]:
sample_cat_df = sample_df['Finding Labels'].str.get_dummies(sep='|').astype('float32')
sample_cat_df.head()

In [ ]:
sample_df.head(10)

### Prepare new csv file with useful information and format

In [ ]:
useful_data_df= sample_df[['Image Index', 'Finding Labels']]
useful_data_df.head()

In [ ]:
len(useful_data_df)

In [ ]:
from collections import Counter
def get_most_common_label(labels):
    return Counter(labels.split('|')).most_common(1)[0][0]

In [ ]:
useful_data_df = useful_data_df.dropna()

In [ ]:
images_df = useful_data_df['Image Index'] 

In [ ]:
images_df

In [ ]:
# Perform stratified split
# X_train, X_test, y_train, y_test = train_test_split(images_df, useful_data_df.primary_label, test_size=0.1, random_state=42, stratify=useful_data_df.primary_label)

### One hot encoding

In [ ]:
# drop_colums = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion',
#        'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding',
#        'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']
# 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion',

labels =['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion',
       'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass',
       'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']

In [ ]:
train_cat_labels_df = useful_data_df['Finding Labels'].str.get_dummies(sep='|').astype('float32')
train_cat_labels_df.columns

In [ ]:
# train_cat_labels_df.drop(['No Finding'], inplace=True, axis=1)
# labels = train_cat_labels_df.columns
train_cat_labels_df = train_cat_labels_df[labels]
train_cat_labels_df.columns

In [ ]:
valid_images = valid_df.Image
valid_labels_df = valid_df[labels]
valid_labels_df.columns

### Load Training Dataset from Dataframe

In [ ]:
def load_image(image_name, label):
    full_path = tf.strings.join([f'{IMAGE_DIR}/', image_name])
    image = tf.io.read_file(full_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])  # Resize to the desired size
    label = tf.cast(label, tf.float32)
    return image, label

def load_image_valid(image_name, label):
    full_path = tf.strings.join(['datasets/images-small/', image_name])
    image = tf.io.read_file(full_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])  # Resize to the desired size
    label = tf.cast(label, tf.float32)
    return image, label


In [ ]:
len(images_df), len(train_cat_labels_df.values)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((images_df,  train_cat_labels_df.values))
train_ds = train_ds.map(load_image, num_parallel_calls=AUTOTUNE)

In [ ]:
valid_ds = tf.data.Dataset.from_tensor_slices((valid_images.values,  valid_labels_df.values))
valid_ds = valid_ds.map(load_image_valid, num_parallel_calls=AUTOTUNE)

## Image Processing

#### Normalization

In [ ]:
# Create normalization layer
normalization_layer = tf.keras.layers.Normalization()
# Compute the mean and variance using the training data
# We need to convert the dataset to numpy to compute statistics
images_for_stats = []
for images, _ in train_ds.take(int(len(images_df)*0.25)): 
    images_for_stats.append(images)
images_for_stats = tf.concat(images_for_stats, axis=0)
normalization_layer.adapt(images_for_stats)

#### Augmentation 

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.10),                    # Small rotation
    tf.keras.layers.RandomTranslation(0.05, 0.05),           # Translation
    tf.keras.layers.RandomContrast(0.1),                     # Contrast adjustment
    tf.keras.layers.RandomBrightness(0.1),                   # Brightness adjustment
    tf.keras.layers.RandomZoom(0.1, 0.1)])                   # Random zoom

In [ ]:
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32) #/ 255.0
    # image = tf.keras.applications.densenet.preprocess_input(image)
    image = normalization_layer(image)
    return image, label

In [ ]:
train_ds = train_ds.map(lambda image, label: (data_augmentation(image), label) , num_parallel_calls=AUTOTUNE)
train_ds = train_ds.map(preprocess_image,num_parallel_calls=AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE).shuffle(len(images_df))
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
for batch in train_ds.take(1):
    print(batch[0].shape)
    image, label = batch
    print(image[0], label[0])

In [ ]:
valid_ds = valid_ds.map(preprocess_image,num_parallel_calls=tf.data.AUTOTUNE)
valid_ds = valid_ds.batch(BATCH_SIZE)
valid_ds = valid_ds.prefetch(buffer_size=AUTOTUNE)

### Class Imbalance Handling

In [ ]:
plt.xticks(rotation=90)
plt.bar(x=labels, height=np.mean(train_cat_labels_df.values, axis=0))
plt.title("Frequency of Each Class")
plt.grid()
plt.show()

#### Compute Class Frequencies

In [ ]:
N = train_cat_labels_df.shape[0]
positive_frequencies = (train_cat_labels_df==1).sum()/N
negative_frequencies = (train_cat_labels_df==0).sum()/N
positive_frequencies, negative_frequencies

In [ ]:
data_df = pd.DataFrame(list(positive_frequencies.items()), columns=['class', 'positives'])
data_df['negatives'] = negative_frequencies.values
data_df

In [ ]:
data_df.plot.bar(x='class')

As we see in the above plot, the contributions of positive cases is significantly lower than that of the negative ones. However, we want the contributions to be equal. One way of doing this is by multiplying each example from each class by a class-specific weight factor, $w_{pos}$ and $w_{neg}$, so that the overall contribution of each class is the same. 

To have this, we want 

$$w_{pos} \times freq_{p} = w_{neg} \times freq_{n},$$

which we can do simply by taking 

$$w_{pos} = freq_{neg}$$
$$w_{neg} = freq_{pos}$$

This way, we will be balancing the contribution of positive and negative labels.

In [ ]:
pos_weights = negative_frequencies.values
neg_weights = positive_frequencies.values
# positive_frequencies.values

# Try adjusting the weight balance slightly
# pos_weights = np.sqrt(negative_frequencies.values) * 0.8  # Reduce positive weight slightly
# neg_weights = np.sqrt(positive_frequencies.values) * 1.2  # Increase negative weight slightly

In [ ]:
pos_contirbution = positive_frequencies * pos_weights
neg_contribution = negative_frequencies * neg_weights

pos_contirbution, neg_contribution

In [ ]:
weighted_data_df = pd.DataFrame(list(pos_contirbution.items()), columns=['class', 'positives'])
weighted_data_df['negatives'] = neg_contribution.values
weighted_data_df

In [ ]:
weighted_data_df.plot.bar(x='class')

## Weighted loss calculation to handle class imbalance

As the above figure shows, by applying these weightings the positive and negative labels within each class would have the same aggregate contribution to the loss function. Now let's implement such a loss function. 

After computing the weights, our final weighted loss for each training case will be 

$$\mathcal{L}_{cross-entropy}^{w}(x) = - (w_{p} y \log(f(x)) + w_{n}(1-y) \log( 1 - f(x) ) ).$$

In [ ]:
def get_weighted_loss(pos_weights, neg_weights, epsilon=1e-7):
    """
    Return weighted loss function given negative weights and positive weights.

    Args:
      pos_weights (np.array): array of positive weights for each class, size (num_classes)
      neg_weights (np.array): array of negative weights for each class, size (num_classes)
    
    Returns:
      weighted_loss (function): weighted loss function
    """
    def weighted_loss(y_true, y_pred):
        """
        Return weighted loss value. 

        Args:
            y_true (Tensor): Tensor of true labels, size is (num_examples, num_classes)
            y_pred (Tensor): Tensor of predicted labels, size is (num_examples, num_classes)
        Returns:
            loss (float): overall scalar loss summed across all classes
        """
        # initialize loss to zero
        loss = 0.0

        for i in range(len(pos_weights)):
            y = y_true[:, i]
            f_of_x = y_pred[:, i]

            f_of_x_log = K.log(f_of_x + epsilon)
            f_of_x_1_min_log = K.log((1-f_of_x) + epsilon)

            first_term = pos_weights[i] * y * f_of_x_log
            sec_term = neg_weights[i] * (1-y) * f_of_x_1_min_log
            loss_per_col = - K.mean(first_term + sec_term)
            loss += loss_per_col
        return loss

    return weighted_loss



## Model Development

### Load and Prepare DenseNet121 Model

In [ ]:
#'imagenet',

base_model = DenseNet121(
     include_top=False,
     weights='pretrain_weights/densenet.hdf5', #'imagenet', 
     input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)  
)
# base_model.trainable = False


x = base_model.output

# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(4096, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
# and a logistic layer
predictions = Dense(len(labels), activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='AUC'), 
]

In [ ]:
tf.keras.backend.clear_session()

LEARNING_RATE = 0.1
EPOCHS = 50
# model = build_model()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), 
               loss=get_weighted_loss(pos_weights, neg_weights),
        metrics=METRICS)     

# model.summary()

## Model training

## Callbacks

In [ ]:
CHECK_POINT_DIR = 'exported_models'
checkpoint_prefix = os.path.join(CHECK_POINT_DIR, "ckpt_{epoch}")
LOG_DIR = 'logs'

In [ ]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True,
                                        save_best_only=True,
                                        monitor='val_loss',
                                        mode='min'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
]

In [ ]:
history = model.fit(train_ds, 
                    validation_data=valid_ds,
                    epochs = EPOCHS,
                    callbacks=callbacks)

## Fine-Tune the Model
Right now, our model is sort of smart. We'll change the learning rate so it doesn't jump to conclusions too quickly.

In [ ]:
# base_model.trainable = True
# fine_tune_at = 149 

# # Freeze all layers before the fine_tune_at layer
# for layer in base_model.layers[:fine_tune_at]:
#     layer.trainable = False

# # Recompile the model with a lower learning rate
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
#     loss=get_weighted_loss(pos_weights, neg_weights),
#     metrics=METRICS
# )

# # Set the number of epochs for fine-tuning
# fine_tune_epochs = 20
# total_epochs = EPOCHS + fine_tune_epochs  # Total epochs

# # Continue training the model with fine-tuning
# history_fine = model.fit(
#     train_ds,
#     epochs=total_epochs,
#     initial_epoch=history.epoch[-1],  # Start from the last epoch of initial training
#     validation_data=valid_ds,
#     callbacks=callbacks  
# )

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train_loss', 'val_loss'])
plt.ylabel("loss")
plt.xlabel("epoch")
plt.title("Training Loss Curve")
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'])
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.title("Training Accuracy Curve")
plt.show()